# Keras을 이용한 전이 학습

https://www.kaggle.com/amadeus1996/fruits-360-transfer-learning-using-keras

이 노트북은 Fruits-360 데이터 세트에 대한 전이학습을 간략하게 적용한 것입니다.

이 데이터 세트는 64개 과일의 42345개의 이미지로 구성됩니다.

우리는 전이 학습 접근 방식을 일반 접근 방식과 비교합니다.

정확도는 2 epoch 내에서 98.44% 입니다.

### Contents
    1. 전이 학습에 대한 간략한 설명
    2. 캐글 커널을 사용한 전이 학습
    3. 데이터 로드
    4. 모델 구축 및 컴파일
    5. 사전교육 모델 학습 및 검증
    6. 일반 모델 학습 및 검증
    7. 두 모델 비교

In [1]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.


## Transfer Learning
전이 학습에서 우리는 먼저 기본 데이터 세트와 작업을 기반으로 기본 네트워크를 훈련한 다음 학습된 피처의 용도를 변경하거나 두 번째 대상 네트워크로 전송하여 대상 데이터 세트와 작업에 대해 훈련합니다. 이 프로세스는 피처가 기본 태스크에 한정되지 않고 기본 태스크와 대상 태스크 모두에 적합한 일반적인 경우 작동하는 경향이 있습니다.

Lisa Torrey와 Jude Shavlik은 전이 학습을 사용할 때 다음과 같은 세 가지 이점을 설명합니다.
   - 그래프가 더 높이 시작합니다. 소스 모델의 초기 기술(모델을 다듬기 전)이 그렇지 않은 경우보다 높습니다.
   - 높은 경사를 가집니다. 소스 모델을 교육하는 동안 스킬의 향상 속도는 그렇지 않은 경우보다 더 가파릅니다.
   - 더 높은 점근선을 가집니다. 훈련된 모델의 융합된 기술은 그렇지 않은 경우보다 더 우수합니다.
   
기본적으로 사전 훈련된 모델(다른 사람에 의해 대규모 데이터 세트에서 훈련된 네트워크의 가중치와 매개 변수)을 취하고 자체 데이터 세트로 모델을 '세밀하게 조정'합니다. 이 사전 훈련된 모델은 초기화된 가중치를 제공하여 더 빠른 수렴을 유도하거나 관심 작업의 고정 피처 추출기로서의 역할을 할 것이라는 생각입니다.

이러한 두 가지 주요 전이 학습 시나리오는 다음과 같습니다.

- convnet 미세 조정: 랜덤 초기화 대신 imagenet 1000 같은 대규모 데이터 세트에서 사전 학습된 네트워크로 네트워크를 초기화합니다. 나머지 훈련은 평소와 같습니다. 이 시나리오에서는 전체 네트워크를 관심있는 데이터 세트에 대해 재교육해야 합니다.

- ConvNet을 고정 피처 추출기로 사용: 여기서는 최종적으로 완전히 연결된 레이어를 제외한 모든 네트워크의 가중치를 동결합니다. 최종적으로 완전히 연결된 레이어는 무작위 가중치를 가진 새로운 레이어로 대체되고 이 레이어만 훈련됩니다.

이 노트북에서는 첫 번째 시나리오를 시연합니다.

## Transfer Learning using Kaggle Kernels
__Using the Keras Pertrained Models dataset__

캐글 커널은 네트워크 연결을 사용하여 사전 훈련된 keras 모델을 다운로드 할 수 없습니다. 이 데이터 세트는 캐글 커널 환경에서 가장 선호하는 사전 훈련된 모델을 사용하는 데 도움이 됩니다.

우리가 해야 할 일은 미리 훈련된 모델을 keras가 갖고 있는 캐시 디렉토리(~/.keras/models)에 복사하는 것입니다. (생략)

In [2]:
cache_dir = expanduser(join('~', '.keras'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)
    
!cp ../input/keras-pretrained-models/*notop* ~/.keras/models/
!cp ../input/keras-pretrained-models/imagenet_class_index.json ~/.keras/models/
!cp ../input/keras-pretrained-models/resnet50* ~/.keras/models/

print('Available Pretrained Models:\n')
!ls ~/.keras/models

'cp'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'cp'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


Available Pretrained Models:



'cp'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'ls'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


## Reading and Visualizing the Data

ImageDataGenerator를 이용하여 데이터를 가져와 네트워크에 제공할 것입니다.

keras는 ImageDataGenerator를 제공하여 이미지 데이터 준비 및 증강을 위한 구성을 정의합니다. 전체 이미지 데이터 세트에 대한 작업을 메모리에 수행하는 대신, API는 딥러닝 모델 피팅 프로세스를 통해 반복되도록 설계되어 있어 적시에 증강 이미지 데이터를 생성할 수 있습니다. 이렇게 하면 메모리 오버헤드가 감소하지만 모델 교육동안 약간의 시간 비용이 추가됩니다.

데이터 생성기 자체는 실제로 반복자로, 요청 시 디렉토리에서 이미지 샘플 배치를 반환합니다. flow_from_directory() 함수를 호출하여 배치 크기를 구성하고 데이터 생성기를 준비하고 이미지 배치를 가져올 수 있습니다.

In [3]:
# dimensions of our images
img_width, img_height = 224, 224 # 우리가 사용할 미리 훈련된 모델에 따라 설정. ResNet-50 입력 사이즈는 224 X 224 X 3

train_data_dir = './input/Training/'
validation_data_dir = './input/Test/'
nb_train_samples = 31688
nb_validation_samples = 10657
batch_size=16

In [4]:
train_datagen = ImageDataGenerator(rescale=1. /255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./ 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(img_height, img_width),
                                                   batch_size=batch_size,
                                                   class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                       target_size=(img_height, img_width),
                                                       batch_size=batch_size,
                                                       class_mode='categorical')

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


### Visualizing the Data

In [5]:
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go
init_notebook_mode(connected=True)

In [6]:
training_data = pd.DataFrame(train_generator.classes, columns=['classes'])
testing_data = pd.DataFrame(validation_generator.classes, columns=['classes'])


In [7]:
def create_stack_bar_data(col, df):
    aggregated = df[col].value_counts().sort_index()
    x_values = aggregated.index.tolist()
    y_values = aggregated.values.tolist()
    return x_values, y_values

In [8]:
x1, y1 = create_stack_bar_data('classes', training_data)
x1 = list(train_generator.class_indices.keys())

trace1 = go.Bar(x=x1, y=y1, opacity=0.75, name='Class Count')
layout = dict(height=400, width=1200, title='Class Distribution in Training Data', legend=dict(orientation='h'), yaxis=dict(title='Class Count'))
fig = go.Figure(data=[trace1], layout=layout)
iplot(fig)

In [9]:
x1, y1 = create_stack_bar_data('classes', testing_data)
x1 = list(validation_generator.class_indices.keys())

trace1 = go.Bar(x=x1, y=y1, opacity=0.75, name='Class Count')
layout = dict(height=400, width=1200, title='Class Distribution in Testing Data', legend=dict(orientation='h'), yaxis=dict(title='Class Count'))
fig = go.Figure(data=[trace1], layout=layout)
iplot(fig)

두 데이터셋의 균형이 잘 이루어져 있습니다.

## Building and Compiling the Model

ResNet-50을 ImageNet 가중치와 함께 로드합니다. 우리는 클래스의 수에 따라 레이어를 추가할 수 있도록 꼭대기 제거합니다. 그런 다음 자체 계층을 추가하여 모델 아키텍처를 완성합니다.

### Pretained Model 구축

In [10]:
inception_base = applications.ResNet50(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = inception_base.output
x = GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = Dense(512, activation='relu')(x)
# and a fully connected output/classification layer
predictions = Dense(131, activation='softmax')(x)
# create the full network so we can train on it
inception_transfer = Model(inputs=inception_base.input, outputs=predictions)

C:\ProgramData\Anaconda3\envs\tens_2g\lib\site-packages\keras_applications\resnet50.py:265: UserWarning:

The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.



### 바닐라 모델 구축

In [11]:
inception_base_vanila = applications.ResNet50(weights=None, include_top = False)

x = inception_base_vanila.output
x = GlobalAveragePooling2D()(x)

x = Dense(512, activation='relu')(x)

predictions = Dense(131, activation='softmax')(x)

inception_transfer_vanila = Model(inputs=inception_base_vanila.input, outputs=predictions)

### Compiling the Models

손실함수, 사용할 최적화 알고리즘 및 각 에포치에 계산할 메트릭을 설정합니다.

In [12]:
inception_transfer.compile(loss='categorical_crossentropy',
                          optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                          metrics=['accuracy'])

inception_transfer_vanila.compile(loss='categorical_crossentropy',
                          optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                          metrics=['accuracy'])


In [13]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12946951022531812366
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4994957312
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4989163198140385765
physical_device_desc: "device: 0, name: GeForce GTX 1660 SUPER, pci bus id: 0000:06:00.0, compute capability: 7.5"
]


## Training and Validating the Pretrained Model

ImageDataGenerateor 클래스의 객체를 사용하여 데이터를 가져오므로 fit_generater() 함수를 이용합니다.

In [14]:

history_pretrained = inception_transfer.fit_generator(train_generator,
                                                         epochs=5, shuffle=True,
                                                         verbose=1, validation_data=validation_generator)

Epoch 1/5
4231/4231 [==============================] - 1409s 333ms/step - loss: 1.1881 - accuracy: 0.8009 - val_loss: 0.2472 - val_accuracy: 0.9762
Epoch 2/5
4231/4231 [==============================] - 1411s 333ms/step - loss: 0.0789 - accuracy: 0.9920 - val_loss: 0.0105 - val_accuracy: 0.9851
Epoch 3/5
4231/4231 [==============================] - 1408s 333ms/step - loss: 0.0361 - accuracy: 0.9969 - val_loss: 0.0422 - val_accuracy: 0.9898
Epoch 4/5
4231/4231 [==============================] - 1390s 329ms/step - loss: 0.0231 - accuracy: 0.9981 - val_loss: 0.0035 - val_accuracy: 0.9894
Epoch 5/5
4231/4231 [==============================] - 1377s 325ms/step - loss: 0.0167 - accuracy: 0.9986 - val_loss: 0.0032 - val_accuracy: 0.9910


In [15]:

history_vanila = inception_transfer_vanila.fit_generator(train_generator,
                                                         epochs=5, shuffle=True,
                                                         verbose=1, validation_data=validation_generator)

Epoch 1/5
4231/4231 [==============================] - 1382s 327ms/step - loss: 3.8444 - accuracy: 0.1370 - val_loss: 2.1908 - val_accuracy: 0.4315
Epoch 2/5
4231/4231 [==============================] - 1367s 323ms/step - loss: 1.7500 - accuracy: 0.5421 - val_loss: 0.8325 - val_accuracy: 0.8426
Epoch 3/5
4231/4231 [==============================] - 1369s 324ms/step - loss: 0.8610 - accuracy: 0.7659 - val_loss: 0.2978 - val_accuracy: 0.8981
Epoch 4/5
4231/4231 [==============================] - 1367s 323ms/step - loss: 0.5139 - accuracy: 0.8619 - val_loss: 0.4162 - val_accuracy: 0.9413
Epoch 5/5
4231/4231 [==============================] - 1365s 323ms/step - loss: 0.3506 - accuracy: 0.9056 - val_loss: 0.0652 - val_accuracy: 0.9572


In [17]:
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history_pretrained.history['val_acc'])
plt.plot(history_vanila.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Pretrained', 'Vanila'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_pretrained.history['val_loss'])
plt.plot(history_vanila.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Pretrained', 'Vanila'], loc='upper left')
plt.show()

KeyError: 'val_acc'